In [ ]:
%matplotlib notebook
from typing import Callable, Dict, List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf

from matplotlib import cm

sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'grid.linestyle':'--'})

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
auto = pd.read_csv("data.csv")
# drop the null values
auto.head()


,X,Y,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,-15.000,-356.996,NaN,NaN,NaN,NaN,NaN
1,-14.824,-394.723,NaN,NaN,NaN,NaN,NaN
2,-14.648,-326.049,NaN,NaN,NaN,NaN,NaN
3,-14.472,-250.838,NaN,NaN,NaN,NaN,NaN
4,-14.296,-373.231,NaN,NaN,NaN,NaN,NaN


In [ ]:
def linear_regression_loss(
    X: np.ndarray,
    y: np.ndarray,
    betas: np.ndarray,
    normalized: bool = True,
) -> float:
    """Calculate the loss of a linear regression problem."""
    if not isinstance(betas, np.ndarray):
        betas = np.array(betas).reshape(-1, 1)
    if y.ndim == 1:
        y = y.reshape(-1, 1)
    
    y_pred = X @ betas
    assert y.shape == y_pred.shape, f"y has shape of {y.shape} and y_pred of {y_pred.shape}"
    #print(y_pred,y)
    loss = np.sum(np.square(y - X @ betas))
    
    if normalized:
        loss /= len(y)
    
    return loss

In [ ]:
def linear_regression_loss_gradient(
    X: np.ndarray,
    y: np.ndarray,
    betas: np.ndarray,
    normalized: bool = True,
) -> np.ndarray:
    """Calculates the gradient of the loss of a linear regression problem."""
    if not isinstance(betas, np.ndarray):
        betas = np.array(betas).reshape(-1, 1)
    if y.ndim == 1:
        y = y.reshape(-1, 1)
        
    grad = -2 * (X.T @ (y - X @ betas))
    assert grad.shape == betas.shape, f"The shape of grad is {grad.shape} and betas is {betas.shape}"
    if normalized:
        grad /= len(y)
    
    return grad

In [ ]:
X_auto = np.vstack((np.ones(shape=len(auto)), auto["X"].values.T)).T
y_auto = auto["Y"].values

# test run
auto_betas = [1, 1]
linear_regression_loss(X=X_auto, y=y_auto, betas=auto_betas)

30068.440558989998

In [ ]:
def gradient_descent(
    X: np.ndarray,
    y: np.ndarray,
    initial_guess: Union[List, np.ndarray], 
    learning_rate: float,
    loss_function: Callable,
    gradient_function: Callable,
    verbose: bool = False,
    threshold: float = 1e-6,
    #fix_guess: Dict = None,
) -> Tuple[List, List]:
    """Gradient descent routine.
    
    Args:
        X: The input data without labels.
        y: The labels.
        initial_guess: The starting point of the gradient descent. 
        learning_rate: The learning rate of the gradient descent.
        loss_function: Provided loss function, that takes (X, y, parameter) as inputs.
        gradient_function: Provided gradient of the loss function, that takes (X, y, parameter) as inputs.
        verbose: If set to True, print out intermediate results.
        threshold: Absolute value of different in loss, below which is considered converged.
        fix_guess: A dictionay to fix given dimension(s) of the parameter.
        
    Return:
        List: The history of parameters.
        List: The history of the losses.
    """
    guess_current = np.array(initial_guess).reshape(-1, 1)
    #if fix_guess:
        #for k, v in fix_guess.items():
            #guess_current[k] = v
            
    guess_iter = [guess_current]
    losses_iter = [loss_function(X=X, y=y, betas=guess_current)]

    difference = float("inf")
    iteration_count = 0
    while abs(difference) > threshold:
        iteration_count += 1
        guess_next = guess_current - learning_rate * gradient_function(
            X=X, y=y, betas=guess_current) 
        #if fix_guess:
            #for k, v in fix_guess.items():
                #guess_next[k] = v
        guess_iter.append(guess_next)
        
        losses_next = loss_function(X=X, y=y, betas=guess_next)
        difference = losses_next - losses_iter[-1]
        losses_iter.append(losses_next)
        
        # update guess
        guess_current = guess_next
        # to print out intermediate results
        if verbose and iteration_count % 10 == 0:
            print(guess_next, losses_next)
    
    # some datatype processing
    guess_iter: List[List[float]] = list(map(lambda x: list(x.flatten()), guess_iter))
    return guess_iter, losses_iter

In [ ]:
auto_guess_iter, auto_losses_iter = gradient_descent(
    X=X_auto,
    y=y_auto,
    initial_guess=[1, 1], 
    learning_rate=0.001,
    loss_function=linear_regression_loss,
    gradient_function=linear_regression_loss_gradient,
    verbose=True,
    threshold=1e-1,
    #fix_guess={1: -0.2}
)

[[ 0.68556267]
 [14.2338476 ]] 7361.811772602872
[[ 0.10365851]
 [15.36640873]] 7162.605592293876
[[-0.49049943]
 [15.47480364]] 7125.895089657189
[[-1.0755107 ]
 [15.49630589]] 7091.654417231319
[[-1.64975487]
 [15.51022545]] 7058.68951656921
[[-2.21328158]
 [15.52327728]] 7026.94525600324
[[-2.76627829]
 [15.53603379]] 6996.376382680395
[[-3.30894069]
 [15.54854756]] 6966.939375425137
[[-3.84146184]
 [15.5608271 ]] 6938.592324911375
[[-4.36403123]
 [15.57287714]] 6911.294873582087
[[-4.87683485]
 [15.58470197]] 6885.008158189488
[[-5.38005519]
 [15.59630583]] 6859.6947544655995
[[-5.87387136]
 [15.60769283]] 6835.3186238414955
[[-6.35845909]
 [15.61886703]] 6811.845062139379
[[-6.83399084]
 [15.62983241]] 6789.240650164399
[[-7.30063586]
 [15.64059287]] 6767.473206125922
[[-7.75856022]
 [15.65115223]] 6746.511739820463
[[-8.2079269 ]
 [15.66151427]] 6726.326408511093
[[-8.6488958 ]
 [15.67168265]] 6706.888474440468
[[-9.08162389]
 [15.68166101]] 6688.1702639170435
[[-9.50626515]
 [15

In [ ]:
auto_losses_iter

In [ ]:
x= auto.X.values
y=auto.Y.values
theto=np.array([[1],[1]])
x=x.reshape((200,1))
y=y.reshape(200,1)
y.shape

(200, 1)

[[-31.80429889]
 [ 16.20562756]]


In [ ]:
def  cal_cost(theta,X,y):
    '''

    
    Calculates the cost for given X and Y. The following shows and example of a single dimensional X
    theta = Vector of thetas 
    X     = Row of X's np.zeros((2,j))
    y     = Actual y's np.zeros((2,1))
    
    where:
        j is the no of features
    '''

    
    m = len(y)
    
    predictions = X @ theta
    cost = (1/(m)) * np.sum(np.square(predictions-y))
    return cost

In [ ]:
cal_cost([1,1],X_b,y)
linear_regression_loss(X_b,y,[1,1])

30068.440558989998

In [ ]:
def minibatch_gradient_descent(X,y,theta,learning_rate=0.01,iterations=10,batch_size =200):
    '''
    X    = Matrix of X without added bias units
    y    = Vector of Y
    theta=Vector of thetas np.random.randn(j,1)
    learning_rate 
    iterations = no of iterations
    
    Returns the final theta vector and array of cost history over no of iterations
    '''
    m = len(y)
    cost_history = np.zeros(iterations)
    n_batches = int(m/batch_size)
    print(theta)
    for it in range(iterations):
        cost =0.0
        #indices = np.random.permutation(m)
        #X = X[indices]
        #y = y[indices]
        for i in range(0,m,batch_size):
            X_i = X[i:i+batch_size]
            y_i = y[i:i+batch_size]
            
            X_i = np.c_[np.ones(len(X_i)),X_i]
           
            prediction = np.dot(X_i,theta)

            theta = theta -(1/m)*learning_rate*( X_i.T.dot((prediction - y_i)))
            cost += linear_regression_loss(X_i,y_i,theta)
        cost_history[it]  = cost
        
        print(theta)
    return theta, cost_history

In [ ]:
minibatch_gradient_descent(x,y,theto,learning_rate=0.001,iterations=10,batch_size =200)

[[1]
 [1]]
[[1.00520977]
 [2.58086123]]
[[1.00646218]
 [3.98882805]]
[[1.00419341]
 [5.24281792]]
[[0.99879195]
 [6.35967841]]
[[0.99060373]
 [7.35441367]]
[[0.97993686]
 [8.24038617]]
[[0.96706573]
 [9.02949628]]
[[0.95223469]
 [9.73234229]]
[[ 0.93566137]
 [10.35836285]]
[[ 0.91753957]
 [10.91596388]]


(array([[ 0.91753957],
        [10.91596388]]),
 array([25343.48451706, 21595.53907114, 18622.4993278 , 16264.06806583,
        14393.10734837, 12908.77915142, 11731.10493065, 10796.65060307,
        10055.10413886,  9466.56111686]))

In [ ]:
import numpy as np 
from PIL import Image

from urllib.request import urlopen

miaw=urlopen('https://raw.githubusercontent.com/changyaochen/MECE4520/master/lectures/lecture_1/leena.png')
image_data = Image.open(miaw)
X = np.array(image_data)

In [ ]:
X

array([[162, 162, 162, ..., 170, 155, 128],
       [162, 162, 162, ..., 170, 155, 128],
       [162, 162, 162, ..., 170, 155, 128],
       ...,
       [ 43,  43,  50, ..., 104, 100,  98],
       [ 44,  44,  55, ..., 104, 105, 108],
       [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8)

In [ ]:
X[128,128]

173

In [ ]:
#performing sscaling only for thee 128th row:
a=X[:,128];
amean=np.mean(a);astd=np.std(a)
ascaled=(a-amean)/(astd)
ascaled[128]

1.2794041765058435

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled[128,128]

1.2794041765058435

In [ ]:
# calculate the mean of each column


In [ ]:
n = len(X_scaled)
U, S, Vh = np.linalg.svd(X_scaled.T @ X_scaled / n)
U[0,0]

-0.05842033740661645

In [ ]:
#all_errors = []
#for k in range(1, len(U) + 1):
k=50
U_reduced = U[:, :k]
Z = X_scaled @ U_reduced
X_approx = Z @ U_reduced.T
error = (
    np.sum(np.square(np.linalg.norm((X_scaled - X_approx), ord=2, axis=1))) 
    / np.sum(np.square(np.linalg.norm(X_scaled, ord=2, axis=1)))
)
    #all_errors.append(error)

#all_errors
error

0.046444313764157766

In [ ]:
k=50
U_reduced = U[:, :k]
U_reduced.shape
X.shape

(512, 512)